In [ ]:
# ! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
set_seed(42)

#### 보이스피싱

- 일단 대화 데이터
- 전자상거래 데이터 

In [ ]:
normal_ver1_data = pd.read_csv("./data/voice_phishing/normal_text_ver1.csv")
normal_ver2_data = pd.read_csv("./data/voice_phishing/normal_text.csv")
phishing_data = pd.read_csv('./data/voice_phishing/phishing_text.csv', encoding='Cp949')

In [ ]:
normal_ver1_data.shape

(73359, 2)

In [ ]:
normal_ver2_data.shape

(71758, 2)

In [ ]:
phishing_data.shape

(1342, 2)

In [ ]:
# 검증을 위한 샘플데이터 추출
sample_data_ver1 = normal_ver1_data.sample(n=1000, replace=False)
normal_ver1_data = normal_ver1_data.drop(sample_data_ver1.index)

sample_data_ver2 = normal_ver2_data.sample(n=1000, replace=False)
normal_ver2_data = normal_ver2_data.drop(sample_data_ver2.index)

sample_phishing_data = phishing_data.sample(n=200, replace=False)
phishing_data = phishing_data.drop(sample_phishing_data.index)

test_data = pd.concat([sample_data_ver1, sample_data_ver2, sample_phishing_data], ignore_index=True)

In [ ]:
normal_ver1_data.shape

(72359, 2)

In [ ]:
normal_ver2_data.shape

(70758, 2)

In [ ]:
phishing_data.shape

(1142, 2)

In [ ]:
data_ver1 = pd.concat([normal_ver1_data, phishing_data])
data_ver2 = pd.concat([normal_ver2_data, phishing_data])
data_ver3 = pd.concat([normal_ver1_data, normal_ver2_data, phishing_data])

In [ ]:
data_ver1.head()

,text,label
0,안녕하세요 일찍 출근하시네요,0
1,그래서 아침밥은 포기하고 삼십 분 일찍 출근하기로 했어요,0
2,아침밥은 회사 앞 편의점에서 해결해도 되니까요,0
3,애들 학원비를 좀 줄이자고 했더니 아내가 펄펄 뛰더라구요,0
4,그렇다고 부모님 용돈을 안 드릴 수도 없고 고민이에요,0


In [ ]:
data_ver2.head()

,text,label
0,고객님 주문은 어떤 방식으로 하시는 건가요,0
1,카드 결제로 가능한가요,0
2,통화 종료 후 #@전번# 번호로 전화가 가면요,0
3,카드 정보를 입력하시면 됩니다,0
4,무이자 할부도 가능한가요,0


In [ ]:
data_ver1 = data_ver1.rename(columns={'text': 'document'})
data_ver2 = data_ver2.rename(columns={'text': 'document'})
data_ver3 = data_ver3.rename(columns={'text': 'document'})
test_data = test_data.rename(columns={'text': 'document'})

In [ ]:
train, test = train_test_split(data_ver3, test_size=0.2, random_state=42)

In [ ]:
# # 데이터 불러오기
# train = pd.read_csv("nsmc/ratings_train.txt",sep="\t").sample(10000)
# test = pd.read_csv("nsmc/ratings_test.txt",sep="\t").sample(2000)

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

In [ ]:
tokenizer

ElectraTokenizer(name_or_path='monologg/koelectra-small-v3-discriminator', vocab_size=35000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
## Data Tokenize 및 Pytorch 형태 변환
max_length = 128

#학습 데이터셋 배치 데이터 인코딩
train_batch_encoding = tokenizer.batch_encode_plus([(row["document"],None) for ix,row in train.iterrows()]
                      ,max_length=max_length
                      ,add_special_tokens=True
                      ,truncation=True
                      ,return_tensors="pt"
                      ,padding="max_length")

test_batch_encoding = tokenizer.batch_encode_plus([(row["document"],None) for ix,row in test.iterrows()]
                      ,max_length=max_length
                      ,add_special_tokens=True
                      ,truncation=True
                      ,return_tensors="pt"
                      ,padding="max_length")



test_check_batch_encoding = tokenizer.batch_encode_plus([(row["document"],None) for ix,row in test_data.iterrows()]
                      ,max_length=max_length
                      ,add_special_tokens=True
                      ,truncation=True
                      ,return_tensors="pt"
                      ,padding="max_length")

In [ ]:
#y데이터 설정
train_labels = torch.tensor(train["label"].tolist(),dtype=torch.long)
test_labels = torch.tensor(test["label"].tolist(),dtype=torch.long)

test_check_labels = torch.tensor(test_data["label"].tolist(),dtype=torch.long)

In [ ]:
train_labels.shape

torch.Size([115407])

In [ ]:
test_labels.shape

torch.Size([28852])

In [ ]:
#입력 데이터셋 구성
train_ds = TensorDataset(train_batch_encoding.input_ids
                         ,train_batch_encoding.attention_mask
                        ,train_batch_encoding.token_type_ids,train_labels)
test_ds = TensorDataset(test_batch_encoding.input_ids
                         ,test_batch_encoding.attention_mask
                        ,test_batch_encoding.token_type_ids,test_labels)
test_check_ds = TensorDataset(test_check_batch_encoding.input_ids
                         ,test_check_batch_encoding.attention_mask
                        ,test_check_batch_encoding.token_type_ids,test_check_labels)

In [ ]:
# torch.save  torch.load 기능을 사용하여 tokenize 된 데이터를 save 하거나 load 가능함
# save
torch.save(train_ds,"cached_nsmc_train")
torch.save(test_ds,"cached_nsmc_test")
torch.save(test_check_ds,"cached_nsmc_test_check")

In [ ]:
# load
train_ds = torch.load("cached_nsmc_train")
test_ds = torch.load("cached_nsmc_test")

test_check_ds = torch.load("cached_nsmc_test_check")

In [ ]:
# Pre-Trained ELECTRA 모델 불러오기
# ElectraForSequenceClassification 모듈은 ELECTRA 모델을 classification으로 fine-tune 할 때 사용하는 모듈
torch.cuda.empty_cache()
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator"
                                                        ,num_labels=train["label"].nunique())

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## 전체 파라미터 수 및 모델 정보 확인
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("ELECTRA-small model 파라미터 수 : ",count_parameters(model))
print("-------------------------------------------------------------------------")
print(model)

ELECTRA-small model 파라미터 수 :  14122498
-------------------------------------------------------------------------
ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout

In [ ]:
# 학습 관련 파라미터
epochs = 3
lr_rate = 5e-05
save_steps = 1000
batch_size = 32
weights_dir = "weights/nsmc/"

if os.path.isdir(weights_dir)==False:
    os.makedirs(weights_dir)

In [ ]:
# train Data Loader
train_sampler = RandomSampler(train_ds) # Data Shuffle,random batch
train_dataloader = DataLoader(train_ds,sampler=train_sampler,batch_size=batch_size)

# Valid Data Loader
test_sampler = SequentialSampler(test_ds) # shuffle 없이 순서대로 데이처 추출
test_dataloader = DataLoader(test_ds,sampler=test_sampler,batch_size=batch_size*2, shuffle=False)

# Valid Data Loader
test_check_sampler = SequentialSampler(test_check_ds) # shuffle 없이 순서대로 데이처 추출
test_check_dataloader = DataLoader(test_check_ds,sampler=test_check_sampler,batch_size=batch_size*2, shuffle=False)


# Optimizer 정의
optimizer = optim.Adam(params = model.parameters() ,lr=lr_rate)

model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

In [ ]:
## 모델 입력 및 결과 확인
batch = next(iter(train_dataloader))
batch = tuple(t.to(device) for t in batch) # cpu 및 gpu 옵션에 따라 데이터 전환
#train mode
inputs = {
    "input_ids": batch[0],
    "attention_mask": batch[1],
    "labels": batch[3]
    }
outputs = model(**inputs)
outputs
# 모델 출력결과는 loss 및 logits으로 구성 됨 (num_labels 2개 이상인 경우 CrossEntropy loss로 적용됨)
# logits 은 모델 forward 예측 결과

SequenceClassifierOutput(loss=tensor(0.6851, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 1.0661e-02, -4.6591e-03],
        [ 1.1513e-02, -4.1519e-03],
        [ 1.6133e-02, -1.2165e-02],
        [ 1.1863e-02, -3.6113e-03],
        [ 4.8675e-03, -9.9896e-03],
        [ 1.2698e-02, -6.4527e-03],
        [ 1.1545e-02, -1.2101e-03],
        [ 2.0849e-02, -3.7688e-03],
        [ 1.8534e-02, -1.9834e-03],
        [ 1.1633e-02, -1.3586e-03],
        [ 1.3303e-02, -3.4699e-03],
        [ 1.6740e-02, -5.2835e-03],
        [ 1.7171e-02, -1.2533e-03],
        [ 9.8002e-03, -4.6701e-03],
        [ 1.1628e-02, -1.1144e-02],
        [ 2.2201e-02, -1.8113e-03],
        [ 9.6321e-03, -6.1825e-03],
        [ 5.4474e-03, -4.5595e-03],
        [ 1.8427e-02, -5.3282e-03],
        [ 1.3566e-02, -1.0607e-02],
        [ 6.9149e-03,  1.2988e-03],
        [ 1.9511e-02, -5.5404e-03],
        [ 1.8503e-02, -1.0928e-02],
        [-8.4685e-03,  4.8488e-03],
        [ 1.2709e-02, -2.3325e-03],
  

In [ ]:
global_step = 0
train_loss = 0.0

model.zero_grad() # gradient 초기화
#train 시작
for epoch in range(epochs):
    print(f"########### Training Epoch:{epoch:02d} ###########")
    for step,batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch) # cpu 및 gpu 옵션에 따라 데이터 전환
        model.train() #train mode
        #입력 변수 파라미터 정의
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids":batch[2],
            "labels": batch[3]
            }
        #모델에 입력된 값을 기준으로 예측값 생성
        outputs = model(**inputs) # 가변 입력 함수

        #예측값과 실제값가지로 로스 계산
        loss = outputs.loss

        loss.backward() # paramter gradient 계산
        if global_step%100==0:
            print(f"Global Step:{global_step:06d}   Loss:{loss.item():.4f}")
        train_loss += loss

        optimizer.step()  # weight 갱신
        model.zero_grad() # gradient 초기화

        #모델 evaluation
        if (global_step%save_steps==0)&(global_step>1):
            print(f"########### Running evalution on {global_step:06d} ###########")
            eval_losses = 0.0
            true_label = []
            pred = []
            eval_step = 0
            for batch in tqdm(test_dataloader):
                model.eval() # evaluation mode
                with torch.no_grad():
                    batch = tuple(t.to(device) for t in batch)
                    inputs = {
                        "input_ids": batch[0],
                        "attention_mask": batch[1],
                        "token_type_ids":batch[2],
                        "labels": batch[3]
                        }
                    outputs = model(**inputs)
                    eval_loss,eval_logits = outputs.loss,outputs.logits

                    eval_losses += eval_loss.mean().item()
                eval_step += 1

                pred.append(outputs.logits.detach().cpu().numpy()) # detach 는 logits의 텐서를 gradient 전파 없이 복사하는 방법
                true_label.append(batch[3].detach().cpu().numpy())

            eval_losses = eval_losses/eval_step

            pred = np.argmax(np.concatenate(pred,axis=0),axis=1)
            true_label = np.concatenate(true_label,axis=0)
            #정확도 계산
            eval_result = (pred==true_label).mean()
            print(f"Evaluation Result Loss:{eval_losses:.4f} acc:{eval_result:.4f}")

            #모델 저장 디렉토리 설정
            output_dir = os.path.join(weights_dir,f"checkpoint-{global_step}")
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            #모델 저장
            model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )
            model_to_save.save_pretrained(output_dir)
            print(f"Saving model checkpoint to {output_dir}")
        global_step += 1
print("Complete Train!!")

########### Training Epoch:00 ###########
Global Step:000000   Loss:0.6869
Global Step:000100   Loss:0.0665
Global Step:000200   Loss:0.0175
Global Step:000300   Loss:0.0108
Global Step:000400   Loss:0.1360
Global Step:000500   Loss:0.1603
Global Step:000600   Loss:0.0050
Global Step:000700   Loss:0.0040
Global Step:000800   Loss:0.0031
Global Step:000900   Loss:0.1866
Global Step:001000   Loss:0.0023
########### Running evalution on 001000 ###########


100%|██████████| 451/451 [00:34<00:00, 13.05it/s]


Evaluation Result Loss:0.0155 acc:0.9964
Saving model checkpoint to weights/nsmc/checkpoint-1000
Global Step:001100   Loss:0.0016
Global Step:001200   Loss:0.0020
Global Step:001300   Loss:0.0049
Global Step:001400   Loss:0.0050
Global Step:001500   Loss:0.0042
Global Step:001600   Loss:0.0016
Global Step:001700   Loss:0.0072
Global Step:001800   Loss:0.0009
Global Step:001900   Loss:0.0008
Global Step:002000   Loss:0.0005
########### Running evalution on 002000 ###########


100%|██████████| 451/451 [00:32<00:00, 13.67it/s]


Evaluation Result Loss:0.0109 acc:0.9980
Saving model checkpoint to weights/nsmc/checkpoint-2000
Global Step:002100   Loss:0.0017
Global Step:002200   Loss:0.0006
Global Step:002300   Loss:0.0004
Global Step:002400   Loss:0.0012
Global Step:002500   Loss:0.0005
Global Step:002600   Loss:0.0010
Global Step:002700   Loss:0.0009
Global Step:002800   Loss:0.0007
Global Step:002900   Loss:0.0005
Global Step:003000   Loss:0.0008
########### Running evalution on 003000 ###########


...

Evaluation Result Loss:0.0051 acc:0.9992
Saving model checkpoint to weights/nsmc/checkpoint-8000
Global Step:008100   Loss:0.0001
Global Step:008200   Loss:0.0001
Global Step:008300   Loss:0.0004
Global Step:008400   Loss:0.0002
Global Step:008500   Loss:0.0001
Global Step:008600   Loss:0.0267
Global Step:008700   Loss:0.0006
Global Step:008800   Loss:0.0001
Global Step:008900   Loss:0.0004
Global Step:009000   Loss:0.0001
########### Running evalution on 009000 ###########


100%|██████████| 451/451 [00:33<00:00, 13.43it/s]


Evaluation Result Loss:0.0036 acc:0.9991
Saving model checkpoint to weights/nsmc/checkpoint-9000
Global Step:009100   Loss:0.0001
Global Step:009200   Loss:0.0005
Global Step:009300   Loss:0.0001
Global Step:009400   Loss:0.0023
Global Step:009500   Loss:0.0002
Global Step:009600   Loss:0.0009
Global Step:009700   Loss:0.0001
Global Step:009800   Loss:0.0029
Global Step:009900   Loss:0.0007
Global Step:010000   Loss:0.0001
########### Running evalution on 010000 ###########


100%|██████████| 451/451 [00:33<00:00, 13.58it/s]


Evaluation Result Loss:0.0043 acc:0.9992
Saving model checkpoint to weights/nsmc/checkpoint-10000
Global Step:010100   Loss:0.0001
Global Step:010200   Loss:0.0002
Global Step:010300   Loss:0.0002
Global Step:010400   Loss:0.0001
Global Step:010500   Loss:0.0029
Global Step:010600   Loss:0.1627
Global Step:010700   Loss:0.0002
Global Step:010800   Loss:0.0001
Complete Train!!


In [ ]:
# Restore Checkpoint and Evaluate
import glob
import re

torch.cuda.empty_cache() #  Reset cuda memory


check_num = re.compile("checkpoint-\d{1,10}")
checkpoints = glob.glob(weights_dir+"/*")
checkpoints = sorted(checkpoints,key=lambda x:int(check_num.findall(x)[0].split("-")[-1]))
for checkpoint in checkpoints:
    global_step = int(check_num.findall(checkpoint)[0].split("-")[-1])
    model = ElectraForSequenceClassification.from_pretrained(checkpoint)
    model.to(device)
    eval_losses = 0.0
    true_label = []
    pred = []
    eval_step = 0
    for batch in tqdm(test_dataloader):
        model.eval() # evaluation mode
        with torch.no_grad():
            batch = tuple(t.to(device) for t in batch)
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids":batch[2],
                "labels": batch[3]
                }
            outputs = model(**inputs)
            eval_loss,eval_logits = outputs.loss,outputs.logits

            eval_losses += eval_loss.mean().item()
        eval_step += 1

        pred.append(outputs.logits.detach().cpu().numpy()) # detach 는 logits의 텐서를 gradient 전파 없이 복사하는 방법
        true_label.append(batch[3].detach().cpu().numpy())

    eval_losses = eval_losses/eval_step

    pred = np.argmax(np.concatenate(pred,axis=0),axis=1)
    true_label = np.concatenate(true_label,axis=0)
    eval_result = (pred==true_label).mean()
    print(f"Global Step:{global_step} Evaluation Result Loss:{eval_losses:.4f} acc:{eval_result:.4f}")
    print(classification_report(true_label, pred))
    print(confusion_matrix(true_label, pred))

100%|██████████| 451/451 [00:33<00:00, 13.56it/s]


Global Step:1000 Evaluation Result Loss:0.0155 acc:0.9964
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28620
           1       0.99      0.56      0.72       232

    accuracy                           1.00     28852
   macro avg       0.99      0.78      0.86     28852
weighted avg       1.00      1.00      1.00     28852

[[28619     1]
 [  102   130]]


100%|██████████| 451/451 [00:33<00:00, 13.55it/s]


Global Step:2000 Evaluation Result Loss:0.0109 acc:0.9980
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28620
           1       1.00      0.75      0.86       232

    accuracy                           1.00     28852
   macro avg       1.00      0.88      0.93     28852
weighted avg       1.00      1.00      1.00     28852

[[28620     0]
 [   58   174]]


100%|██████████| 451/451 [00:33<00:00, 13.60it/s]


...

Global Step:8000 Evaluation Result Loss:0.0051 acc:0.9992
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28620
           1       1.00      0.90      0.95       232

    accuracy                           1.00     28852
   macro avg       1.00      0.95      0.97     28852
weighted avg       1.00      1.00      1.00     28852

[[28619     1]
 [   23   209]]


100%|██████████| 451/451 [00:33<00:00, 13.63it/s]


Global Step:9000 Evaluation Result Loss:0.0036 acc:0.9991
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28620
           1       0.97      0.92      0.94       232

    accuracy                           1.00     28852
   macro avg       0.98      0.96      0.97     28852
weighted avg       1.00      1.00      1.00     28852

[[28613     7]
 [   18   214]]


100%|██████████| 451/451 [00:33<00:00, 13.58it/s]

Global Step:10000 Evaluation Result Loss:0.0043 acc:0.9992
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28620
           1       0.97      0.92      0.95       232

    accuracy                           1.00     28852
   macro avg       0.99      0.96      0.97     28852
weighted avg       1.00      1.00      1.00     28852

[[28614     6]
 [   18   214]]


ver1
```
100%|██████████| 230/230 [00:16<00:00, 13.58it/s]
Global Step:1000 Evaluation Result Loss:0.0094 acc:0.9983
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14473
           1       0.96      0.93      0.94       228

    accuracy                           1.00     14701
   macro avg       0.98      0.97      0.97     14701
weighted avg       1.00      1.00      1.00     14701

100%|██████████| 230/230 [00:16<00:00, 13.60it/s]
Global Step:2000 Evaluation Result Loss:0.0085 acc:0.9976
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14508
           1       0.85      0.98      0.91       193

    accuracy                           1.00     14701
   macro avg       0.93      0.99      0.96     14701
weighted avg       1.00      1.00      1.00     14701

100%|██████████| 230/230 [00:16<00:00, 13.60it/s]
Global Step:3000 Evaluation Result Loss:0.0016 acc:0.9995
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14485
           1       0.97      1.00      0.98       216

    accuracy                           1.00     14701
   macro avg       0.98      1.00      0.99     14701
weighted avg       1.00      1.00      1.00     14701

100%|██████████| 230/230 [00:16<00:00, 13.59it/s]
Global Step:4000 Evaluation Result Loss:0.0015 acc:0.9998
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14477
           1       1.00      0.99      0.99       224

    accuracy                           1.00     14701
   macro avg       1.00      1.00      1.00     14701
weighted avg       1.00      1.00      1.00     14701

100%|██████████| 230/230 [00:16<00:00, 13.65it/s]Global Step:5000 Evaluation Result Loss:0.0026 acc:0.9995
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14486
           1       0.96      1.00      0.98       215

    accuracy                           1.00     14701
   macro avg       0.98      1.00      0.99     14701
weighted avg       1.00      1.00      1.00     14701

```



ver1_check



```
100%|██████████| 35/35 [00:02<00:00, 14.85it/s]
Global Step:1000 Evaluation Result Loss:0.6663 acc:0.7959
              precision    recall  f1-score   support

           0       0.78      0.99      0.87      1571
           1       0.95      0.30      0.46       629

    accuracy                           0.80      2200
   macro avg       0.87      0.65      0.67      2200
weighted avg       0.83      0.80      0.76      2200

100%|██████████| 35/35 [00:02<00:00, 14.95it/s]
Global Step:2000 Evaluation Result Loss:0.5159 acc:0.8773
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      1814
           1       0.79      0.41      0.54       386

    accuracy                           0.88      2200
   macro avg       0.84      0.69      0.73      2200
weighted avg       0.87      0.88      0.86      2200

100%|██████████| 35/35 [00:02<00:00, 14.34it/s]
Global Step:3000 Evaluation Result Loss:1.0722 acc:0.7945
              precision    recall  f1-score   support

           0       0.78      0.99      0.87      1568
           1       0.95      0.30      0.46       632

    accuracy                           0.79      2200
   macro avg       0.86      0.65      0.67      2200
weighted avg       0.83      0.79      0.75      2200

100%|██████████| 35/35 [00:02<00:00, 15.25it/s]
Global Step:4000 Evaluation Result Loss:1.8594 acc:0.6732
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      1289
           1       0.98      0.22      0.35       911

    accuracy                           0.67      2200
   macro avg       0.81      0.61      0.57      2200
weighted avg       0.78      0.67      0.60      2200

100%|██████████| 35/35 [00:02<00:00, 15.17it/s]Global Step:5000 Evaluation Result Loss:1.3998 acc:0.7895
              precision    recall  f1-score   support

           0       0.77      0.99      0.87      1561
           1       0.94      0.29      0.45       639

    accuracy                           0.79      2200
   macro avg       0.86      0.64      0.66      2200
weighted avg       0.82      0.79      0.75      2200


```



ver2



```
100%|██████████| 225/225 [00:16<00:00, 13.62it/s]
Global Step:1000 Evaluation Result Loss:0.0084 acc:0.9983
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14188
           1       0.90      0.98      0.94       192

    accuracy                           1.00     14380
   macro avg       0.95      0.99      0.97     14380
weighted avg       1.00      1.00      1.00     14380

100%|██████████| 225/225 [00:16<00:00, 13.60it/s]
Global Step:2000 Evaluation Result Loss:0.0075 acc:0.9984
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14178
           1       0.93      0.96      0.94       202

    accuracy                           1.00     14380
   macro avg       0.96      0.98      0.97     14380
weighted avg       1.00      1.00      1.00     14380

100%|██████████| 225/225 [00:16<00:00, 13.61it/s]
Global Step:3000 Evaluation Result Loss:0.0062 acc:0.9987
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14183
           1       0.93      0.98      0.96       197

    accuracy                           1.00     14380
   macro avg       0.96      0.99      0.98     14380
weighted avg       1.00      1.00      1.00     14380

100%|██████████| 225/225 [00:16<00:00, 13.65it/s]
Global Step:4000 Evaluation Result Loss:0.0080 acc:0.9987
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14182
           1       0.93      0.98      0.95       198

    accuracy                           1.00     14380
   macro avg       0.96      0.99      0.98     14380
weighted avg       1.00      1.00      1.00     14380

100%|██████████| 225/225 [00:16<00:00, 13.59it/s]Global Step:5000 Evaluation Result Loss:0.0135 acc:0.9971
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14150
           1       0.95      0.87      0.91       230

    accuracy                           1.00     14380
   macro avg       0.97      0.93      0.95     14380
weighted avg       1.00      1.00      1.00     14380
```



ver2_check


```
100%|██████████| 35/35 [00:02<00:00, 13.97it/s]
Global Step:1000 Evaluation Result Loss:0.1896 acc:0.9223
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1875
           1       0.89      0.54      0.67       325

    accuracy                           0.92      2200
   macro avg       0.91      0.77      0.82      2200
weighted avg       0.92      0.92      0.91      2200

100%|██████████| 35/35 [00:02<00:00, 13.87it/s]
Global Step:2000 Evaluation Result Loss:0.3855 acc:0.9118
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      1836
           1       0.93      0.51      0.66       364

    accuracy                           0.91      2200
   macro avg       0.92      0.75      0.80      2200
weighted avg       0.91      0.91      0.90      2200

100%|██████████| 35/35 [00:02<00:00, 14.61it/s]
Global Step:3000 Evaluation Result Loss:0.2631 acc:0.9273
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1878
           1       0.91      0.56      0.69       322

    accuracy                           0.93      2200
   macro avg       0.92      0.78      0.83      2200
weighted avg       0.93      0.93      0.92      2200

100%|██████████| 35/35 [00:02<00:00, 14.69it/s]
Global Step:4000 Evaluation Result Loss:0.3276 acc:0.9232
              precision    recall  f1-score   support

           0       0.92      0.99      0.96      1859
           1       0.93      0.55      0.69       341

    accuracy                           0.92      2200
   macro avg       0.93      0.77      0.82      2200
weighted avg       0.92      0.92      0.91      2200

100%|██████████| 35/35 [00:02<00:00, 15.28it/s]Global Step:5000 Evaluation Result Loss:0.5052 acc:0.8982
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      1796
           1       0.95      0.47      0.63       404

    accuracy                           0.90      2200
   macro avg       0.92      0.73      0.79      2200
weighted avg       0.90      0.90      0.88      2200
```



ver3


```
100%|██████████| 451/451 [00:33<00:00, 13.55it/s]
Global Step:1000 Evaluation Result Loss:0.0147 acc:0.9968
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28697
           1       0.63      0.95      0.76       155

    accuracy                           1.00     28852
   macro avg       0.82      0.97      0.88     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.51it/s]
Global Step:2000 Evaluation Result Loss:0.0084 acc:0.9980
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28621
           1       0.87      0.87      0.87       231

    accuracy                           1.00     28852
   macro avg       0.93      0.94      0.94     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.66it/s]
Global Step:3000 Evaluation Result Loss:0.0075 acc:0.9984
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28636
           1       0.87      0.93      0.90       216

    accuracy                           1.00     28852
   macro avg       0.93      0.96      0.95     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.56it/s]
Global Step:4000 Evaluation Result Loss:0.0099 acc:0.9981
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28668
           1       0.78      0.98      0.87       184

    accuracy                           1.00     28852
   macro avg       0.89      0.99      0.93     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.62it/s]
Global Step:5000 Evaluation Result Loss:0.0087 acc:0.9984
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28638
           1       0.86      0.93      0.90       214

    accuracy                           1.00     28852
   macro avg       0.93      0.97      0.95     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.64it/s]
Global Step:6000 Evaluation Result Loss:0.0077 acc:0.9986
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28645
           1       0.86      0.96      0.91       207

    accuracy                           1.00     28852
   macro avg       0.93      0.98      0.95     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.59it/s]
Global Step:7000 Evaluation Result Loss:0.0056 acc:0.9989
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28637
           1       0.90      0.97      0.93       215

    accuracy                           1.00     28852
   macro avg       0.95      0.98      0.97     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.55it/s]
Global Step:8000 Evaluation Result Loss:0.0171 acc:0.9964
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28724
           1       0.55      1.00      0.71       128

    accuracy                           1.00     28852
   macro avg       0.78      1.00      0.85     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.58it/s]
Global Step:9000 Evaluation Result Loss:0.0065 acc:0.9986
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28622
           1       0.91      0.92      0.91       230

    accuracy                           1.00     28852
   macro avg       0.95      0.96      0.96     28852
weighted avg       1.00      1.00      1.00     28852

100%|██████████| 451/451 [00:33<00:00, 13.66it/s]Global Step:10000 Evaluation Result Loss:0.0059 acc:0.9989
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28640
           1       0.89      0.97      0.93       212

    accuracy                           1.00     28852
   macro avg       0.94      0.99      0.96     28852
weighted avg       1.00      1.00      1.00     28852
```



- 성능 확인

In [ ]:
df = pd.read_csv('./date/stt.csv')


In [ ]:
df

,document,target
0,네 여보세요. 여보세요. 은혁 씨 점심 드셨어요.,0
1,네 먹었어요. 어디서 드셨어요. 저기 밖에 월드컵 경기장으로 갔습니다.,1
2,저는 점심 조금 늦게 먹을 것 같아 가지고 먼저 들어가세요.,0
3,알겠습니다. 네 어쩔 수 없죠. 그럼 커피는 좀 드시나요. 네 그냥 아메리카노 사서...,1
4,알겠습니다.,0


추론함수 구현

In [ ]:
checkpoint = "./weights/nsmc/checkpoint-7000"
model = ElectraForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
#학습 데이터셋 배치 데이터 인코딩
sample = tokenizer.batch_encode_plus(["안녕하세요? 고객님"]
                      ,max_length=max_length
                      ,add_special_tokens=True
                      ,truncation=True
                      ,return_tensors="pt"
                      ,padding="max_length")

sample_ds = TensorDataset(sample.input_ids
                         ,sample.attention_mask
                        ,sample.token_type_ids)

In [ ]:
sample_loader = DataLoader(sample_ds)

In [ ]:
model = ElectraForSequenceClassification.from_pretrained(checkpoint)
model.to(device)
pred = []
for batch in tqdm(sample_loader):
    model.eval() # evaluation mode
    with torch.no_grad():
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids":batch[2]
            }
        outputs = model(**inputs)
        eval_logits = outputs.logits

    pred.append(outputs.logits.detach().cpu().numpy())

pred = np.argmax(np.concatenate(pred,axis=0),axis=1)

100%|██████████| 1/1 [00:00<00:00, 21.96it/s]


In [ ]:
pred

array([0])

In [ ]:
def ELEC_Classifier(input_point, text):
  checkpoint = input_point
  model = ElectraForSequenceClassification.from_pretrained(checkpoint)
  #학습 데이터셋 배치 데이터 인코딩
  sample = tokenizer.batch_encode_plus([text]
                        ,max_length=max_length
                        ,add_special_tokens=True
                        ,truncation=True
                        ,return_tensors="pt"
                        ,padding="max_length")

  sample_ds = TensorDataset(sample.input_ids
                          ,sample.attention_mask
                          ,sample.token_type_ids)

  sample_loader = DataLoader(sample_ds)
  model = ElectraForSequenceClassification.from_pretrained(checkpoint)
  model.to(device)
  pred = []
  for batch in tqdm(sample_loader):
      model.eval() # evaluation mode
      with torch.no_grad():
          batch = tuple(t.to(device) for t in batch)
          inputs = {
              "input_ids": batch[0],
              "attention_mask": batch[1],
              "token_type_ids":batch[2]
              }
          outputs = model(**inputs)
          eval_logits = outputs.logits

      pred.append(outputs.logits.detach().cpu().numpy())

  pred = np.argmax(np.concatenate(pred,axis=0),axis=1)
  print(" ")
  if pred == 1:
    print(" 보이스피싱입니다.")
  elif pred == 0:
    print(" 일반통화입니다.")
  else:
    print("?")

In [ ]:
ELEC_Classifier("/content/weights/nsmc/checkpoint-7000",
                text=input())

알겠습니다. 네 어쩔 수 없죠. 그럼 커피는 좀 드시나요. 네 그냥 아메리카노 사서


100%|██████████| 1/1 [00:00<00:00, 41.38it/s]

 
 일반통화입니다.
